In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from read_data import get_dirs
import os

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image

In [2]:
class PhotoDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row["path"]).convert("RGB")
        
        if self.transform:
            image = self.transform(image)

        label = torch.tensor(row["label"], dtype=torch.long)
        return image, label

In [3]:
transform = transforms.Compose([
    # change image size
    transforms.Resize((224, 224)),
    # converts PIL image to PyTorch tensor
    transforms.ToTensor()
])


In [4]:
df = get_dirs()

# initialising every image transformed
dataset = PhotoDataset(df, transform=transform)

# data wrapper for convenient usage to model
train_loader = DataLoader(dataset, batch_size=20, shuffle=True)

# 20 images/labels, 3 colors, 224 height, 224 width
# 20 at a time
images, labels = next(iter(train_loader))
print(images.shape, labels.shape)



torch.Size([20, 3, 224, 224]) torch.Size([20])


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using:", device)

# resnet18 model with default weights (pre-trained IMAGENET1K_V1)
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# X*W + c, using final layer nodes to output ???
model.fc = nn.Linear(model.fc.in_features, 2)   # Alex vs Kelly
model = model.to(device)

Using: cpu


In [6]:
criterion = nn.CrossEntropyLoss()

# optimizer on weights using learning rate 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [7]:
batch_losses = []
batch_accs = []

model.train()

for batch_idx, (imgs, lbls) in enumerate(train_loader):
    imgs, lbls = imgs.to(device), lbls.to(device)

    # clear old gradients
    optimizer.zero_grad()

    # output of model
    outputs = model(imgs)

    # calculate loss based on current output/weights
    loss = criterion(outputs, lbls)

    # backward propagation to train model
    loss.backward()

    # forward propagation to update parameters
    optimizer.step()

    preds = outputs.argmax(dim=1)
    acc = (preds == lbls).float().mean().item()

    batch_losses.append(loss.item())
    batch_accs.append(acc)

    print(f"Batch {batch_idx}: Loss = {loss.item():.4f}, Acc = {acc:.3f}")


Batch 0: Loss = 0.8478, Acc = 0.300
Batch 1: Loss = 0.6444, Acc = 0.550
Batch 2: Loss = 0.6700, Acc = 0.600
Batch 3: Loss = 0.6189, Acc = 0.850
Batch 4: Loss = 0.5850, Acc = 0.650
Batch 5: Loss = 0.4242, Acc = 0.800
Batch 6: Loss = 0.6176, Acc = 0.650
Batch 7: Loss = 0.5570, Acc = 0.750
Batch 8: Loss = 0.3753, Acc = 0.850
Batch 9: Loss = 0.4901, Acc = 0.700
Batch 10: Loss = 0.2534, Acc = 0.900
Batch 11: Loss = 0.4394, Acc = 0.850
Batch 12: Loss = 0.4570, Acc = 0.750
Batch 13: Loss = 0.3663, Acc = 0.800
Batch 14: Loss = 0.4753, Acc = 0.850
Batch 15: Loss = 0.3033, Acc = 0.900
Batch 16: Loss = 0.4295, Acc = 0.900
Batch 17: Loss = 0.3472, Acc = 0.850
Batch 18: Loss = 0.5269, Acc = 0.750
Batch 19: Loss = 0.3150, Acc = 0.800
Batch 20: Loss = 0.3339, Acc = 0.800
Batch 21: Loss = 0.5229, Acc = 0.750
Batch 22: Loss = 0.1958, Acc = 1.000
Batch 23: Loss = 0.3396, Acc = 0.850
Batch 24: Loss = 0.0645, Acc = 1.000
